# **PICO EXTRACTION** *| BERT NER*
### Participants/Problem (P), Intervention (I), Comparison (C) and Outcome (O)

In [ ]:
# Intstall required libraries
!pip install datasets
!pip install transformers
!pip install seqeval

In [24]:
# Import necessary libraries
import re
import pandas as pd
import numpy as np
import itertools
from datasets import Dataset
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [25]:
# Create necessary functions
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

    
def replace_entity(old_entity):
    if "POPULATION" in old_entity or "SAMPLE" in old_entity:
        entity = "I-POPULATION"
    elif "PARTICIPANT" in old_entity:
        entity = "I-PARTICIPANT"
    elif "INTERVENTION" in old_entity:
        entity = "I-INTERVENTION"
    elif "COMPAR" in old_entity:
        entity = "I-COMPARISON"
    elif "OUTCOME" in old_entity:
        entity = "I-OUTCOME"
    elif bool(re.search('[a-zA-Z]', old_entity)) == False:
        entity = ""
    else:
        entity = "O"
    return(entity)

In [ ]:
# Download the PubMed data
!wget https://raw.githubusercontent.com/janarishsaju/pico_extraction/main/pubmed_data.txt

In [27]:
# Read the text data
with open("pubmed_data.txt") as f:
    content = f.readlines()

### **DATA PREPARATION**
Convert the text data to PICO BERT trainable data format

Convert the text file and seperate the following fields in seperate column in a dataframe

* PMID
* PICO_Entity
* AOMRC Entity
* Text

In [29]:
# Data preprocessing & manipulation
content_df = pd.DataFrame()
for i in range(len(content)):
    if("###" in content[i]):
        content_df.loc[i, "PMID"] = content[i].split(":")[0].replace("#", "")
    elif(content[i] == "\n"):
        pass
    else:
        content_df.loc[i, "PMID"] = content_df.loc[i-1, "PMID"]
        content_df.loc[i, "PICO_Entity"] = content[i].split("|")[0]
        content_df.loc[i, "AOMRC_Entity"] = content[i].split("|")[1]
        content_df.loc[i, "Text"] = content[i].split("|")[2]

In [30]:
# Display manipulated data | samples
content_df.head()

,PMID,PICO_Entity,AOMRC_Entity,Text
0,28628768,NaN,NaN,NaN
1,28628768,OBJECTIVE,A,To test the efficacy of a pregnancy adapted ve...
2,28628768,SETTING,M,Online and telephone .\n
3,28628768,POPULATION OR SAMPLE,P,Self-referred pregnant women ( gestational wee...
4,28628768,METHODS,M,@ pregnant women ( gestational week @ ) with m...


### **DATA CLEANING**
You can there are NAN's occupied with the columns

In [31]:
# Remove NAN's
content_df = content_df[pd.notnull(content_df['Text'])]

In [32]:
# Display cleaned data | samples
content_df.head()

,PMID,PICO_Entity,AOMRC_Entity,Text
1,28628768,OBJECTIVE,A,To test the efficacy of a pregnancy adapted ve...
2,28628768,SETTING,M,Online and telephone .\n
3,28628768,POPULATION OR SAMPLE,P,Self-referred pregnant women ( gestational wee...
4,28628768,METHODS,M,@ pregnant women ( gestational week @ ) with m...
5,28628768,MAIN OUTCOME MEASURES,O,The primary outcome was depressive symptoms me...


### **DATA CONVERSION**
Convert the data with the required format and rename

In [ ]:
# Datatype formating and renaming
content_df["Text"] = content_df["Text"].astype(str)
content_df["Text"] = content_df["Text"].apply(lambda x:x.strip())
content_df = content_df[["PMID", "Text", "PICO_Entity"]]
content_df.rename(columns = {'Text':'tokens', 'PICO_Entity':'ner_tags'}, inplace = True)
content_df = content_df.reset_index(drop=True)

In [34]:
# Display formatted samples
content_df.head()

,PMID,tokens,ner_tags
0,28628768,To test the efficacy of a pregnancy adapted ve...,OBJECTIVE
1,28628768,Online and telephone .,SETTING
2,28628768,Self-referred pregnant women ( gestational wee...,POPULATION OR SAMPLE
3,28628768,@ pregnant women ( gestational week @ ) with m...,METHODS
4,28628768,The primary outcome was depressive symptoms me...,MAIN OUTCOME MEASURES


### **DATA PREPROCESSING**
Convert the data required parameters that needs to train the model

In [35]:
# Calculate token length
content_df["token_len"] = content_df["tokens"].apply(lambda x: len(x.split()))

In [36]:
# Display samples
print(content_df.head())

       PMID                                             tokens  \
0  28628768  To test the efficacy of a pregnancy adapted ve...   
1  28628768                             Online and telephone .   
2  28628768  Self-referred pregnant women ( gestational wee...   
3  28628768  @ pregnant women ( gestational week @ ) with m...   
4  28628768  The primary outcome was depressive symptoms me...   

                ner_tags  token_len  
0              OBJECTIVE         29  
1                SETTING          4  
2   POPULATION OR SAMPLE         17  
3                METHODS         36  
4  MAIN OUTCOME MEASURES         18  


In [37]:
# Replace entities with required entity names
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x: replace_entity(x))

In [38]:
# Display samples
content_df.head()

,PMID,tokens,ner_tags,token_len
0,28628768,To test the efficacy of a pregnancy adapted ve...,O,29
1,28628768,Online and telephone .,O,4
2,28628768,Self-referred pregnant women ( gestational wee...,I-POPULATION,17
3,28628768,@ pregnant women ( gestational week @ ) with m...,O,36
4,28628768,The primary outcome was depressive symptoms me...,I-OUTCOME,18


In [40]:
# Iterate entities to all words/token
for i in range(len(content_df)):
    content_df.loc[i, "ner_tags"] = str([content_df["ner_tags"][i]] * content_df["token_len"][i])
    content_df.loc[i, "tokens"] = str(content_df["tokens"][i].split())

In [41]:
content_df = pd.read_excel("df.xlsx")

In [42]:
# Converts string to list of values
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x:eval(x))
content_df["tokens"] = content_df["tokens"].apply(lambda x:eval(x))

In [43]:
# Group by PMID
content_df = content_df.groupby('PMID').agg(lambda x: x.to_list())

In [44]:
# Combine list of list
content_df["tokens"] = content_df["tokens"].apply(lambda x:list(itertools.chain.from_iterable(x)))
content_df["ner_tags"] = content_df["ner_tags"].apply(lambda x:list(itertools.chain.from_iterable(x)))

In [45]:
# Reset dataframe
content_df = content_df.reset_index(drop=True)

In [47]:
# Display preprocessed data | samples
content_df.head()

,Unnamed: 0,tokens,ner_tags,token_len
0,"[313316, 313317, 313318, 313319, 313320, 31332...","[To, determine, whether, prophylactic, treatme...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[25, 38, 9, 19, 15, 53, 49, 21, 29]"
1,"[313133, 313134, 313135, 313136, 313137, 31313...","[Since, it, is, not, clear, whether, testoster...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[44, 20, 38, 17, 17, 10, 21, 7, 25, 10, 30, 24]"
2,"[312675, 312676, 312677, 312678, 312679, 31268...","[The, aim, was, to, study, the, pharmacokineti...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[30, 24, 41, 22, 12, 10, 42, 13, 16, 45, 37, 2..."
3,"[312748, 312749, 312750, 312751, 312752, 31275...","[To, investigate, the, significance, of, treat...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[33, 15, 29, 32, 26, 19, 35, 33, 37, 16, 38, 7]"
4,"[312661, 312662, 312663, 312664, 312665, 31266...","[Dopamine, agonists, have, a, well, establishe...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[26, 34, 12, 45, 22, 45, 27, 65, 22, 47, 23, 3..."


In [48]:
# Save preprocessed data
content_df.to_excel("preprocessed_df.xlsx")

**TEST / TRAIN SPLIT**
---
Split the train data set into two sets by the ratio 8:2. and we can furthur evaluate the model performance

In [49]:
#Test train split
train, test = train_test_split(content_df, test_size=0.2, shuffle=False, random_state=1)

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train_df = train
test_df = test

In [50]:
# Print Shape
print(len(train_df))
print(len(test_df))

19734
4934


In [51]:
# Label to numeric representation
label_list = ['O','I-POPULATION', 'I-PARTICIPANT','I-INTERVENTION', 'I-COMPARISON','I-OUTCOME']
label_encoding_dict = {'O': 0, 'I-POPULATION': 1, 'I-PARTICIPANT': 2, 'I-INTERVENTION': 3, 'I-COMPARISON': 4, 'I-OUTCOME': 5}

**MODEL INITIALIZATION**
---
Initialize the BERT model. Define the Task Name, Model, Tokenizer

In [52]:
# Initialize the BERT model
task = "ner" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# Initialize pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 

data_collator = DataCollatorForTokenClassification(tokenizer) #Some random data enhancements

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

**TOKENIZATION**
---
Tokenize and embed the dataset with pretrained BERT

In [53]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

**HYPER PARAMETER TURNING**
---
Hyperparameters - We can iterate and tune the model with these parameters for better results

In [54]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

metric = load_metric("seqeval")

<ipython-input-54-4245283469b2>:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


**TRAINING MODULE**
---
Train the model with different parameters and finalize the optimal one

In [55]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, Unnamed: 0, token_len, ner_tags. If tokens, Unnamed: 0, token_len, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 19734
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3702
  Number of trainable parameters = 66367494
You're using a DistilBertTokenizerFast tokenizer. Please note th

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.273400,0.312759,0.284052,0.607024,0.387007,0.883307
2,0.212600,0.259378,0.349804,0.637421,0.451716,0.905596
3,0.135200,0.270499,0.371685,0.639033,0.470000,0.908633


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, Unnamed: 0, token_len, ner_tags. If tokens, Unnamed: 0, token_len, ner_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***

TrainOutput(global_step=3702, training_loss=0.21938356845717633, metrics={'train_runtime': 3361.4806, 'train_samples_per_second': 17.612, 'train_steps_per_second': 1.101, 'total_flos': 7636512478492368.0, 'train_loss': 0.21938356845717633, 'epoch': 3.0})

# **EVALUATION**

---

Evaluate the remaining 20 percent of training data. 
It is a good idea to take a part of training data for validation while training the model.

In [30]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, ner_tags, token_len, Unnamed: 0. If tokens, ner_tags, token_len, Unnamed: 0 are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4934
  Batch size = 16


{'eval_loss': 0.27075833082199097,
 'eval_precision': 0.42053466509988247,
 'eval_recall': 0.6592976396085204,
 'eval_f1': 0.5135195731133133,
 'eval_accuracy': 0.9128263240249777,
 'eval_runtime': 127.1476,
 'eval_samples_per_second': 38.805,
 'eval_steps_per_second': 2.43,
 'epoch': 3.0}

*Save the Model*
---
Save the model to disk

In [31]:
# Save the model to disk
trainer.save_model('un-ner.model')

Saving model checkpoint to un-ner.model
Configuration saved in un-ner.model/config.json
Model weights saved in un-ner.model/pytorch_model.bin
tokenizer config file saved in un-ner.model/tokenizer_config.json
Special tokens file saved in un-ner.model/special_tokens_map.json


**PREDICTION MODULE**
---
Predict the **Unseen Data**


In [32]:
# Load the tokenizer and trained model from disk
tokenizer = AutoTokenizer.from_pretrained('./un-ner.model/')
model = AutoModelForTokenClassification.from_pretrained('./un-ner.model/', num_labels=len(label_list))

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file ./un-ner.model/config.json
Model config DistilBertConfig {
  "_name_or_path": "./un-ner.model/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",

In [33]:
# Input unseen data
paragraph = '''
OBJECTIVE|A|To test the efficacy of a pregnancy adapted version of an existing 10-week ICBT-program for depression as well as assessing acceptability and adherence DESIGN: Randomised controlled trial.
SETTING|M|Online and telephone.
POPULATION OR SAMPLE|P|Self-referred pregnant women (gestational week 10-28 at intake) currently suffering from major depressive disorder.
METHODS|M|42 pregnant women (gestational week 12-28) with major depression were randomised to either treatment as usual (TAU) provided at their antenatal clinic or to ICBT as an add-on to usual care.
MAIN OUTCOME MEASURES|O|The primary outcome was depressive symptoms measured with the Montgomery-sberg depression rating scale-self report (MADRS-S). The Edinburgh Postnatal Depression Scale and measures of anxiety and sleep were used. Credibility, satisfaction, adherence and utilization were also assessed.
RESULTS|R|The ICBT group had significantly lower levels of depressive symptoms post treatment (p < 0.001, Hedges g =1.21) and were more likely to be responders (i.e. achieve a statistically reliable improvement) (RR = 0.36; p = 0.004). Measures of treatment credibility, satisfaction, utilization, and adherence were comparable to implemented ICBT for depression.
LIMITATIONS|Others|Small sample size and no long-term evaluation.
CONCLUSION|C|Pregnancy adapted ICBT for antenatal depression is feasible, acceptable and efficacious. These results need to be replicated in larger trials to validate these promising findings.
'''

In [34]:
# Tokenize the paragraph
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

torch.Size([1, 332])

In [35]:
# Predictions
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

In [36]:
print(predictions)

['I-POPULATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PARTICIPANT', 'O', 'I-PARTICIPANT', 'O', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATION', 'I-POPULATI

In [40]:
words = tokenizer.batch_decode(tokens['input_ids'])

In [43]:
# Store the token and predicted entities in a dataframe
prediction_results = pd.DataFrame({'ner': predictions, 'words': words})

In [45]:
# Print predicted samples
print(prediction_results)

              ner      words
0    I-POPULATION      [CLS]
1               O  objective
2               O          |
3               O          a
4               O          |
..            ...        ...
327             O      these
328             O  promising
329             O   findings
330             O          .
331  I-POPULATION      [SEP]

[332 rows x 2 columns]


*Export the results*
---
Export test results to **csv**

Select the **Words, Labels** and **Confidence scores** to export csv file

In [44]:
# Export results to csv
prediction_results.to_csv("prediction_results.csv")